In [ ]:
import re
from typing import List, Optional
from transformers import AutoTokenizer

class DataCleaner:
    def __init__(self,
                 base_model: str = "meta-llama/Meta-Llama-3.1-8B",
                 important_fields: Optional[List[str]] = None,
                 removals: Optional[List[str]] = None,
                 min_chars: int = 300,
                 max_tokens: int = 160,
                 min_tokens: int = 150):
        
        # Tokenizer yükle
        self.tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

        # Hangi alanları kullanacağız
        self.important_fields = important_fields or ["title", "description", "features", "details"]

        # Silinecek gürültü kalıplar
        self.removals = removals or []

        # Limitler
        self.min_chars = min_chars
        self.max_tokens = max_tokens
        self.min_tokens = min_tokens
        self.ceiling_chars = max_tokens * 7  # Yaklaşık token-karakter oranı

    def scrub_text(self, text: str) -> str:
        """
        Genel metin temizliği: 
        - Gereksiz semboller
        - Boşluk düzenleme
        - SKU/parça numaralarını kaldırma
        """
        if not text:
            return ""
        # Gürültü kalıplarını sil
        for pattern in self.removals:
            text = text.replace(pattern, "")

        # Noktalama ve boşluk temizliği
        text = re.sub(r'[:\[\]"{}【】\s]+', ' ', text).strip()
        text = text.replace(" ,", ",").replace(",,,", ",").replace(",,", ",")

        # SKU veya uzun numaraları sil
        words = text.split(' ')
        select = [w for w in words if len(w) < 7 or not any(c.isdigit() for c in w)]
        return " ".join(select)

    def prepare_text(self, record: dict) -> Optional[str]:
        """
        Kaydı temizler ve token limitlerine göre kırpar.
        """
        # Önemli alanları birleştir
        contents = ""
        for field in self.important_fields:
            value = record.get(field)
            if isinstance(value, list):
                value = "\n".join([str(v) for v in value if v])
            if value:
                contents += str(value) + "\n"

        # Karakter kontrolü
        if len(contents) < self.min_chars:
            return None

        # Karakter tavanı
        contents = contents[:self.ceiling_chars]

        # Temizleme
        contents = self.scrub_text(contents)

        # Token sayımı
        tokens = self.tokenizer.encode(contents, add_special_tokens=False)
        if len(tokens) < self.min_tokens:
            return None

        # Token kesme
        tokens = tokens[:self.max_tokens]
        return self.tokenizer.decode(tokens)

    def process_dataset(self, dataset: List[dict]) -> List[dict]:
        """
        Tüm veri setini temizler ve uygun kayıtları döndürür.
        """
        cleaned_records = []
        for record in dataset:
            cleaned_text = self.prepare_text(record)
            if cleaned_text:
                record["cleaned_text"] = cleaned_text
                cleaned_records.append(record)
        return cleaned_records


In [1]:
#nasıl kullanılır
# Temizleyici oluştur
cleaner = DataCleaner(
    important_fields=["title", "description", "specifications"],  # veri setine göre değiştir
    removals=["Best Sellers", "SKU", "Product ID"]                 # veri setine göre değiştir
)

# Veri seti örneği
dataset = [
    {"title": "Super Blender 5000", "description": ["Powerful motor", "Easy to clean"], "specifications": "Color: Red", "price": 129.99},
    {"title": "Cheap Kettle", "description": ["Boils water"], "specifications": "SKU: KTL-500", "price": 9.99}
]

# Temizleme işlemi
cleaned_data = cleaner.process_dataset(dataset)

# Çıktı kontrol
for item in cleaned_data:
    print(item["cleaned_text"])


NameError: name 'DataCleaner' is not defined